# Accessing ML Audio Features from Spotify
---------------
This notebook demonstrates how to access the audio features to be used in random forest classification to automatically come up with a new playlist based on an old playlist!

In [59]:
## Relevant imports
import os
from typing import Union, Iterable

from dotenv import dotenv_values
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn import model_selection, linear_model
import pandas as pd

The first thing we need is a client ID and secret from [Spotify's developer API](). Go through the steps of registering for a developer account and make an app in accordance with [these directions]().

In [2]:
# Set up Spotify API client credentials
config = dotenv_values('.env')
client_id = config["SPOTIFY_CLIENT_ID"]
client_secret = config["SPOTIFY_CLIENT_SECRET"]

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [63]:
class Playlist():

    def __init__(self, playlist_id):
        ## Get track metadata
        self.playlist = sp.playlist(playlist_id=playlist_id)
        self.raw_tracks = self.playlist['tracks']['items']
        self.track_attribute_labels = self.raw_tracks[0]['track'].keys()
        self.length = len(self.raw_tracks)

        ## All useful information
        self.track_ids = self.get_info('id')
        self.track_names = self.get_info('name')
        self.track_artists = self.get_info('artist')
        self.audio_features = sp.audio_features(self.track_ids)
        
        ## Machine Learning-friendly formatting
        self.data = pd.DataFrame(index=[self.track_ids, self.track_names], 
                                 data=self.audio_features)
        self.data.index.names = ['id', 'name']
        self.audio_feature_labels = self.data.columns

        # ML specifically (e.g. random forest)
        self.ml_feature_labels = list(set(self.audio_feature_labels) - \
                {'type', 'id', 'uri', 'track_href', 'analysis_url'})
        self.ml_data = self.data[self.ml_feature_labels]

    def get_info(self, info_tag:str):
        '''Unnest information from raw_tracks dict.'''

        if any([info_tag in track['track'].keys() 
                                            for track in self.raw_tracks]):
            attributes = [track['track'][info_tag] 
                            if info_tag in track['track'].keys()
                            else None
                            for track in self.raw_tracks]
            return attributes
        
    def set_like_status(self, like_labels:Union[Iterable, int]):
        '''Set the like status of songs'''

        if isinstance(like_labels, Iterable):
            if len(like_labels) == self.data.shape[0] and \
                not (isinstance(like_labels, dict) or 
                     isinstance(like_labels, pd.Series)):
                self.data['like'] = like_labels
                self.ml_data['like'] = like_labels
            elif isinstance(like_labels, dict):
                pass
        elif isinstance(like_labels, int):
            # Set all like values of playlist to one number
            likes = [like_labels]*self.data.shape[0]
            self.data['like'] = likes
            self.ml_data['like'] = likes


In [64]:
# Specify the playlist ID for which you want to obtain song IDs and attributes
rb_playlist_id = '5Os91wBXvZCdwiOa2aJV5I' #https://open.spotify.com/playlist/5Os91wBXvZCdwiOa2aJV5I?si=22eadc4636634b6d

# Get the playlist information using the playlist ID
rb_playlist = Playlist(rb_playlist_id)

In [65]:
# https://open.spotify.com/playlist/2q4kES8D0GmKOJx4Uqvliv?si=dfb36f6a589f4335
lnv_playlist_id = '2q4kES8D0GmKOJx4Uqvliv'

lnv_playlist = Playlist(lnv_playlist_id)

In [66]:
rb_playlist.track_attribute_labels

dict_keys(['album', 'artists', 'available_markets', 'disc_number', 'duration_ms', 'episode', 'explicit', 'external_ids', 'external_urls', 'href', 'id', 'is_local', 'name', 'popularity', 'preview_url', 'track', 'track_number', 'type', 'uri'])

In [39]:
# Example of how to get name from id
rb_playlist.data.loc['7dMTCS9BLzBqYTlAuHP8TM'].index.get_level_values('name')

Index(['XT4S1S'], dtype='object', name='name')

In [69]:
rb_playlist.ml_data

,,loudness,valence,speechiness,liveness,time_signature,energy,tempo,acousticness,instrumentalness,mode,duration_ms,key,danceability
id,name,,,,,,,,,,,,,
5awNIWVrh2ISfvPd5IUZNh,PTT (Paint The Town),-2.437,0.546,0.1000,0.0565,4,0.814,124.028,0.07350,0.000019,1,201120,8,0.781
65497v9jzFfu5sSA8ADap5,Sistema De Patio,-2.243,0.694,0.2100,0.3280,4,0.948,109.999,0.07000,0.000782,0,134223,6,0.914
7I0vl97TYZW86qYdgcD1qu,Back Door,-3.853,0.653,0.2720,0.1120,4,0.842,107.952,0.15700,0.000000,1,189287,7,0.820
7dMTCS9BLzBqYTlAuHP8TM,XT4S1S,-4.163,0.919,0.0857,0.4410,4,0.677,130.034,0.10100,0.000000,0,219320,9,0.797
0Z99Xe1lGBmq60RwJ5YU18,PING PONG,-2.059,0.863,0.0826,0.0202,4,0.870,103.028,0.01220,0.000675,0,158253,10,0.855
1t8sqIScEIP0B4bQzBuI2P,MY BAG,-3.929,0.759,0.1640,0.1560,4,0.823,94.024,0.08640,0.000071,0,160520,8,0.838
0bspC5fLlWgwCM6Rx1YUIh,BIG MAD,-6.100,0.309,0.0591,0.1060,4,0.523,102.039,0.02930,0.000000,1,143111,8,0.958
7gRFDGEzF9UkBV233yv2dc,Shut Down,-5.119,0.688,0.0454,0.1860,3,0.690,110.071,0.00327,0.000000,1,175889,0,0.818
4kXxEhuatrvwrTQycA7s9B,BIZCOCHITO,-4.203,0.781,0.0744,0.1200,4,0.774,116.972,0.60500,0.023800,1,109120,1,0.930
